In [7]:
pip install transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 115.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [5]:
pip install transformers datasets


In [10]:
pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.5/416.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 33.3 MB/s eta 0:00:00


In [1]:
from github import Github
import re
import os
from datasets import Dataset

# Method 1: Using environment variable (recommended)
try:
    github_token = os.getenv('GITHUB_TOKEN')
    if not github_token:
        raise ValueError("GITHUB_TOKEN environment variable not set")
    g = Github(github_token)
    print("GitHub connection initialized successfully")
except Exception as e:
    print(f"Error initializing GitHub connection: {e}")
    exit(1)

# Method 2: Alternative - using GitHub CLI authentication (uncomment if needed)
# from github import Auth
# auth = Auth.Token(os.getenv('GITHUB_TOKEN'))
# g = Github(auth=auth)

# specify the repository
try:
    repo = g.get_repo("openai/gym")
    print(f"Successfully accessed repository: {repo.full_name}")
except Exception as e:
    print(f"Error accessing repository: {e}")
    exit(1)

# function to extract Python functions from a script
def extract_functions_from_code(code):
    pattern = re.compile(r"def\s+(\w+)\s*\(.*?\):", re.DOTALL)
    functions = pattern.findall(code)
    return functions

# fetch Python files from the repository with error handling
python_files = []
try:
    contents = repo.get_contents("")
    file_count = 0

    while contents:
        file_content = contents.pop(0)
        if file_content.type == "dir":
            try:
                contents.extend(repo.get_contents(file_content.path))
            except Exception as e:
                print(f"Warning: Could not access directory {file_content.path}: {e}")
                continue
        elif file_content.path.endswith(".py"):
            python_files.append(file_content)
            file_count += 1
            if file_count % 10 == 0:  # Progress indicator
                print(f"Found {file_count} Python files...")

    print(f"Total Python files found: {len(python_files)}")

except Exception as e:
    print(f"Error fetching repository contents: {e}")
    exit(1)

# extract functions and create dataset
data = {"code": [], "function_name": [], "file_path": []}
total_functions = 0

for i, file in enumerate(python_files):
    try:
        print(f"Processing file {i+1}/{len(python_files)}: {file.path}")

        # Handle large files by checking size first
        if file.size > 1000000:  # Skip files larger than 1MB
            print(f"Skipping large file: {file.path} ({file.size} bytes)")
            continue

        code = file.decoded_content.decode("utf-8")
        functions = extract_functions_from_code(code)

        for function in functions:
            data["code"].append(code)
            data["function_name"].append(function)
            data["file_path"].append(file.path)
            total_functions += 1

    except Exception as e:
        print(f"Warning: Could not process file {file.path}: {e}")
        continue

print(f"Total functions extracted: {total_functions}")

if total_functions == 0:
    print("No functions found. Exiting...")
    exit(1)

# create a Hugging Face dataset
try:
    dataset = Dataset.from_dict(data)
    print(f"Dataset created with {len(dataset)} entries")

    # save the dataset to disk
    dataset.save_to_disk("code_generation_dataset")
    print("Dataset created and saved to disk successfully.")

    # Print some statistics
    print(f"\nDataset Statistics:")
    print(f"- Total entries: {len(dataset)}")
    print(f"- Unique files: {len(set(data['file_path']))}")
    print(f"- Sample function names: {list(set(data['function_name']))[:10]}")

except Exception as e:
    print(f"Error creating or saving dataset: {e}")
    exit(1)

Error initializing GitHub connection: GITHUB_TOKEN environment variable not set
Error accessing repository: name 'g' is not defined
Error fetching repository contents: name 'repo' is not defined
Total functions extracted: 0
No functions found. Exiting...
Dataset created with 0 entries


Saving the dataset (0/1 shards): 0 examples [00:00, ? examples/s]

Dataset created and saved to disk successfully.

Dataset Statistics:
- Total entries: 0
- Unique files: 0
- Sample function names: []


In [2]:
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

# load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")
model = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")

# set the pad_token to eos_token or add a new pad token
tokenizer.pad_token = tokenizer.eos_token

# load the dataset
dataset = load_from_disk("code_generation_dataset")

# split the dataset into training and test sets
dataset = dataset.train_test_split(test_size=0.1)

# preprocess the dataset
def preprocess_function(examples):
    return tokenizer(examples['code'], truncation=True, padding='max_length')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/240 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/999 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/797M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/797M [00:00<?, ?B/s]

Some weights of the model checkpoint at Salesforce/codegen-350M-mono were not used when initializing CodeGenForCausalLM: ['transformer.h.0.attn.causal_mask', 'transformer.h.1.attn.causal_mask', 'transformer.h.10.attn.causal_mask', 'transformer.h.11.attn.causal_mask', 'transformer.h.12.attn.causal_mask', 'transformer.h.13.attn.causal_mask', 'transformer.h.14.attn.causal_mask', 'transformer.h.15.attn.causal_mask', 'transformer.h.16.attn.causal_mask', 'transformer.h.17.attn.causal_mask', 'transformer.h.18.attn.causal_mask', 'transformer.h.19.attn.causal_mask', 'transformer.h.2.attn.causal_mask', 'transformer.h.3.attn.causal_mask', 'transformer.h.4.attn.causal_mask', 'transformer.h.5.attn.causal_mask', 'transformer.h.6.attn.causal_mask', 'transformer.h.7.attn.causal_mask', 'transformer.h.8.attn.causal_mask', 'transformer.h.9.attn.causal_mask']
- This IS expected if you are initializing CodeGenForCausalLM from the checkpoint of a model trained on another task or with another architecture (e

In [6]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from datasets import Dataset, load_from_disk
import torch
import os

# First, let's create a proper dataset if the saved one is empty
def create_sample_dataset():
    """Create a sample code dataset for testing"""
    sample_data = {
        "code": [
            "def hello_world():\n    print('Hello, World!')\n    return 'Hello'",
            "def add_numbers(a, b):\n    result = a + b\n    return result",
            "def fibonacci(n):\n    if n <= 1:\n        return n\n    return fibonacci(n-1) + fibonacci(n-2)",
            "def factorial(n):\n    if n == 0:\n        return 1\n    return n * factorial(n-1)",
            "def reverse_string(s):\n    return s[::-1]",
            "def is_prime(n):\n    if n < 2:\n        return False\n    for i in range(2, int(n**0.5) + 1):\n        if n % i == 0:\n            return False\n    return True",
            "def bubble_sort(arr):\n    n = len(arr)\n    for i in range(n):\n        for j in range(0, n-i-1):\n            if arr[j] > arr[j+1]:\n                arr[j], arr[j+1] = arr[j+1], arr[j]\n    return arr",
            "def binary_search(arr, target):\n    left, right = 0, len(arr) - 1\n    while left <= right:\n        mid = (left + right) // 2\n        if arr[mid] == target:\n            return mid\n        elif arr[mid] < target:\n            left = mid + 1\n        else:\n            right = mid - 1\n    return -1"
        ],
        "function_name": [
            "hello_world", "add_numbers", "fibonacci", "factorial",
            "reverse_string", "is_prime", "bubble_sort", "binary_search"
        ]
    }
    return Dataset.from_dict(sample_data)

# Load or create dataset
try:
    if os.path.exists("code_generation_dataset"):
        dataset = load_from_disk("code_generation_dataset")
        print(f"Dataset loaded with {len(dataset)} entries")

        if len(dataset) == 0:
            print("Dataset is empty, creating sample dataset...")
            dataset = create_sample_dataset()
    else:
        print("Dataset not found, creating sample dataset...")
        dataset = create_sample_dataset()

except Exception as e:
    print(f"Error loading dataset: {e}")
    print("Creating sample dataset...")
    dataset = create_sample_dataset()

print(f"Using dataset with {len(dataset)} entries")

# Use a proper code generation model
model_name = "microsoft/DialoGPT-small"  # Better for generation tasks
print(f"Loading model: {model_name}")

try:
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Add padding token if it doesn't exist
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print("Model and tokenizer loaded successfully")

except Exception as e:
    print(f"Error loading model: {e}")
    print("Trying GPT-2 model...")

    # Alternative: use GPT-2
    model_name = "gpt2"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

# Define preprocessing function for code generation
def preprocess_function(examples):
    """
    Preprocess the dataset for code generation fine-tuning
    """
    # Create training texts that combine function name and code
    texts = []

    for i in range(len(examples['function_name'])):
        # Create a format like: "Function: function_name\nCode:\ndef function_name()..."
        text = f"Function: {examples['function_name'][i]}\nCode:\n{examples['code'][i]}<|endoftext|>"
        texts.append(text)

    # Tokenize the texts
    tokenized = tokenizer(
        texts,
        max_length=256,  # Reduced for better memory usage
        truncation=True,
        padding=True,
        return_tensors="pt"
    )

    # For causal language modeling, labels are the same as input_ids
    tokenized["labels"] = tokenized["input_ids"].clone()

    return tokenized

# Apply preprocessing
print("Preprocessing dataset...")
try:
    tokenized_datasets = dataset.map(
        preprocess_function,
        batched=True,
        remove_columns=dataset.column_names
    )
    print("Dataset preprocessing completed")

except Exception as e:
    print(f"Error in preprocessing: {e}")
    # Simple fallback preprocessing
    def simple_preprocess(examples):
        tokenized = tokenizer(
            examples['code'],
            max_length=256,
            truncation=True,
            padding=True
        )
        tokenized["labels"] = tokenized["input_ids"]
        return tokenized

    tokenized_datasets = dataset.map(simple_preprocess, batched=True)

# Split dataset into train/test
print("Splitting dataset into train/test...")
split_datasets = tokenized_datasets.train_test_split(test_size=0.2, seed=42)
print(f"Train size: {len(split_datasets['train'])}, Test size: {len(split_datasets['test'])}")

# Create data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal LM, not masked LM
)

# Define training arguments (using correct parameter names)
training_args = TrainingArguments(
    output_dir="./results",
    overwrite_output_dir=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    save_steps=100,
    save_total_limit=2,
    logging_steps=50,
    eval_strategy="steps",  # Changed from evaluation_strategy
    eval_steps=100,
    warmup_steps=50,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to=[],  # Empty list instead of None
    dataloader_pin_memory=False,
    remove_unused_columns=False,
)

# Initialize trainer
print("Initializing trainer...")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=split_datasets['train'],
    eval_dataset=split_datasets['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Start training
print("Starting training...")
try:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    trainer.train()
    print("Training completed successfully!")

    # Save the model
    trainer.save_model("./fine_tuned_code_model")
    tokenizer.save_pretrained("./fine_tuned_code_model")
    print("Model saved to ./fine_tuned_code_model")

except RuntimeError as e:
    if "out of memory" in str(e).lower():
        print("GPU out of memory! Trying with smaller batch size...")

        # Reduce batch size and try again
        training_args = TrainingArguments(
            output_dir="./results",
            overwrite_output_dir=True,
            per_device_train_batch_size=1,
            per_device_eval_batch_size=1,
            gradient_accumulation_steps=2,  # Maintain effective batch size
            num_train_epochs=2,
            save_steps=50,
            save_total_limit=2,
            logging_steps=25,
            eval_strategy="steps",
            eval_steps=50,
            warmup_steps=25,
            learning_rate=5e-5,
            weight_decay=0.01,
            logging_dir='./logs',
            report_to=[],
            dataloader_pin_memory=False,
            remove_unused_columns=False,
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=split_datasets['train'],
            eval_dataset=split_datasets['test'],
            data_collator=data_collator,
            tokenizer=tokenizer,
        )

        try:
            trainer.train()
            print("Training completed with reduced batch size!")
            trainer.save_model("./fine_tuned_code_model")
            tokenizer.save_pretrained("./fine_tuned_code_model")
        except Exception as e2:
            print(f"Training failed: {e2}")
    else:
        print(f"Training error: {e}")

except Exception as e:
    print(f"Training error: {e}")

# Test the model
print("\n" + "="*50)
print("Testing the fine-tuned model...")
print("="*50)

try:
    # Test generation
    model.eval()
    test_prompts = [
        "Function: square\nCode:\n",
        "Function: max_value\nCode:\n",
        "Function: count_vowels\nCode:\n"
    ]

    for prompt in test_prompts:
        print(f"\nPrompt: {prompt.strip()}")

        inputs = tokenizer.encode(prompt, return_tensors="pt")

        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_length=inputs.shape[1] + 50,
                num_return_sequences=1,
                temperature=0.8,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.encode("<|endoftext|>")[0] if "<|endoftext|>" in tokenizer.get_vocab() else tokenizer.eos_token_id
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Generated: {generated_text}")
        print("-" * 30)

except Exception as e:
    print(f"Error during testing: {e}")

print("\nFine-tuning process completed!")
print(f"Model saved in: ./fine_tuned_code_model")
print(f"You can load it later with:")
print("model = AutoModelForCausalLM.from_pretrained('./fine_tuned_code_model')")
print("tokenizer = AutoTokenizer.from_pretrained('./fine_tuned_code_model')")

Dataset loaded with 0 entries
Dataset is empty, creating sample dataset...
Using dataset with 8 entries
Loading model: microsoft/DialoGPT-small


tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/641 [00:00<?, ?B/s]

...

/tmp/ipython-input-1654674733.py:165: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


Training completed successfully!


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Model saved to ./fine_tuned_code_model

Testing the fine-tuned model...

Prompt: Function: square
Code:
Generated: Function: square
Code:
What's your C assignment?
------------------------------

Prompt: Function: max_value
Code:
Generated: Function: max_value
Code:
Code :
------------------------------

Prompt: Function: count_vowels
Code:
Generated: Function: count_vowels
Code:
count :
------------------------------

Fine-tuning process completed!
Model saved in: ./fine_tuned_code_model
You can load it later with:
model = AutoModelForCausalLM.from_pretrained('./fine_tuned_code_model')
tokenizer = AutoTokenizer.from_pretrained('./fine_tuned_code_model')


In [7]:
# define a function to generate code using the fine-tuned model
def generate_code(prompt, max_length=100):
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(inputs['input_ids'], max_length=max_length)
    generated_code = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_code

# test the model with a code generation prompt
prompt = "def merge_sort(arr):"
generated_code = generate_code(prompt)

print("Generated Code:")
print(generated_code)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Code:
def merge_sort(arr):
